<a href="https://colab.research.google.com/github/Gibbons-Lab/isb_course_2022/blob/main/micom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧫🦠 Modeling microbiota-wide metabolism with MICOM

This notebook will accompany the second session of the 2022 ISB Microbiome Course. The presentation slides can be [found here](https://gibbons-lab.github.io/isb_course_2022/micom). 

You can save your own local copy of this notebook by using `File > Save a copy in Drive`. You may be promted to cetify the notebook is safe. We promise that it is 🤞

**Disclaimer:**
The linear and quadratic programming problems MICOM has to solve are very large and very complicated. There are some very good commercial solvers that are very expensive (even though they are often free for academic use). To make this tutorial as accessible as possible we will use the Open Source solver [OSQP](https://osqp.org/). OSQP is amazing with quadratic programming problems (kudos!) but not as accurate for linear problems. Solvers usually only guarantee a solution within a certain numerical tolerance of the real solution. In order to make everything work with OSQP this tolerance has to be relaxed to about 10<sup>-3</sup>. This means that any result with an absolute value smaller than that might very well be zero so we should look at larger values only. Installing cost-free academic versions of commercial solvers like [IBM CPLEX](https://www.ibm.com/analytics/cplex-optimizer) or [Gurobi](https://www.gurobi.com/) would alow you to lower the tolerance to 10<sup>-6</sup>.



# 📝 Setup

MICOM installation is is usually pretty straight-forward and can be as easy as typing `pip install micom` into your Terminal. However, MICOM will require a solver for quadratic programming problems and all the best ones are commercial (boo) even though they often have free academic licenses 😌. We will use a brand new open source QP solver named OSQP, but this will require us to pull in development versions of certain packages.

But first let's start by downloading the materials again and switching to the folder.

In [1]:
!git clone https://github.com/gibbons-lab/isb_course_2022 materials
%cd materials

Cloning into 'materials'...
remote: Enumerating objects: 772, done.
remote: Counting objects: 100% (383/383), done.
remote: Compressing objects: 100% (255/255), done.
remote: Total 772 (delta 165), reused 307 (delta 121), pack-reused 389
Receiving objects: 100% (772/772), 205.22 MiB | 50.12 MiB/s, done.
Resolving deltas: 100% (254/254), done.
/content/materials


## Basic Installation

Installing MICOM is straight-forward in Python. OSQP itself will be installed automatically along with it.

In [ ]:
!pip install -q micom

print("Done! 🎉 ")

## Enable QIIME 2 interactions

Before we start, we also need to install packages to read the "biom" file format used by QIIME 2 to save tables. This is only necessary if you want to read QIIME 2 FeatureTable artifacts (like the ones we constructed yesterday)

In [ ]:
!pip install -q numpy Cython
!pip install -q biom-format

print("Done! 🎉 ")

Okay, all done. So let's get started building some models 🦺🛠d😁.

# 💻 MICOM

We will use the Python interface to MICOM since it plays nicely with Colaboratory. However, you could run the same steps within the QIIME 2 MICOM plugin ([q2-micom](https://library.qiime2.org/plugins/q2-micom/26/)). 

Here is an overview of all the steps and functions:
![micom overview](https://github.com/micom-dev/q2-micom/raw/706f583a060b91c12c0cec7acea2354fdd0dd320/docs/assets/overview.png).

The process of building a metabolic model in MICOM begins with constructing a a combined abundance/taxonomy, referred to hereafter as a taxonomy table. Let's load a sample taxonomy table to see what it looks like: 



In [ ]:
from micom.data import test_data

test_data().head()

In this sample taxonomy table, we see four identical strains of _E. coli_ (1 through 4), across two samples (sample_1 and sample_2). We can see that each row represents a single taxon in a single sample, and the `abundance` column identifies the abundance of that taxon in the sample. 

The `id` column specifies identifiers for the taxa and should be expressive and not include spaces or special characters. Since we are using a taxonomy database to build our models (more on that soon), we don't need a `file` column.

You might notice that this dataframe looks very different from what we generated in yesterday's tutorial, where we ended up with separate QIIME 2 artifacts 😱 

No worries, we can deal with that.

## Importing data from QIIME 2

MICOM can read QIIME 2 artifacts. You don't even need to have QIIME 2 installed for that! But before we do so, let's resolve one issue. We discussed that MICOM summarizes genome-scale models into pangenome-scale models as a first step, but our data are on the ASV level...so how will we know what to summarize?

Basically, a specific model database can be used to quickly summarize pangenome-scale models for use within MICOM. So, before we read our data we have to decide which model database to use. We will go with the [AGORA database](https://pubmed.ncbi.nlm.nih.gov/27893703/), which is a curated database of more than 800 bacterial strains that commonly live in the human gut. In particular, we will use a version of this database summarized on the genus rank which can be downloaded from the [MICOM data repository](https://doi.org/10.5281/zenodo.3755182), which contains a whole lot of prebuilt databases.

In [ ]:
!wget -O agora103_genus.qza https://zenodo.org/record/3755182/files/agora103_genus.qza?download=1

Now we're all set to start building models! The data we previously collected can be found in the `treasure_chest` folder, so we can use those files to build our taxonomy for MICOM. 

In [ ]:
from micom.taxonomy import qiime_to_micom

tax = qiime_to_micom(
    "treasure_chest/dada2/table.qza", 
    "treasure_chest/taxa.qza", 
    collapse_on="genus"
)

Notice the `collapse_on` argument. That will specify the rank on which to sumarize and can be a list of several ranks. When matching taxonomy you can either match by the particular rank of interest (for example, just comparing genus names here), or you could compare the entire taxonomy, which will require all taxonomic ranks prior to the target rank to match. For that you cloud specify `collapse_on=["kingdom", "phylum", "class", "order", "family", "genus"]`. 

Taxonomic names will often not match 100% between databases. For instance, the genus name "Prevotella" in one database may be "Prevotella_6" in another. The more ranks you use for matching the more likely are you to run into those issues. However, the more taxonomic ranks you use to match the more confident you can be that your observed taxon really is the same taxon as the one in the model database.

The resulting table will contain the same abundances but it will include more ranks if `collapse_on` is a list. All ranks present in the taxonomy will be used when matching to the database. The GreenGenes database is pretty old and many taxonomic names have been superceded by now. So we will stick with the "lax" option of only matching on genus ranks.

Let's now take a look at the taxonomy table we generated:

In [ ]:
tax

That looks more like the example! Again, we have a row for each taxon in each sample, so we're good to go. 

One helpful thing to do is to merge in our metadata, so we'll have it at hand for the following steps. In our case, the metadata will include the ethnic group, region, and subsistence type of each of the study participants. 

In [ ]:
import pandas as pd

metadata = pd.read_table("data/metadata.tsv").rename(columns={"id": "sample_id"})
tax = pd.merge(tax, metadata, on="sample_id")
tax

With our taxonmy table ready to go, and our metadata merged, its finally time to get to the model building! 🎉

## Building community models

With the data we have now, building our models is pretty easy. We just pass our taxonomy table and model database to MICOM. We will remove all taxa that make up less than 1% of the community to keep the models small and speed up this tutorial. We will also have to specify where to write the models. For simplicity, we'll run this process in parallel over two threads. It should take around 10 minutes to finish.

In [ ]:
from micom.workflows import build
from micom import Community
import pandas as pd

manifest = build(tax, "agora103_genus.qza", "models", solver="osqp", 
                 cutoff=0.01, threads=2)

In [ ]:
manifest

You'll see a warning pop up indicating that less than 50% of the abundances can be matched to the database. This can happen with some data, and may indicate the models may not be completely representative of the samples. We'll continue, but remember to keep an eye out for this in future projects!

In lower-biomass 16S amplicon sequencing samples from stool, many reads can match to food components or to host mitochondria and these hits probably do not contribute much to bacterial community metabolism. These hits will be excluded from MICOM. 

Let's also take a look what we got back from the `build` process.

In [ ]:
manifest

This will tell you many taxa were found in the database and what fraction of the total abundance was represented by the database. Looks okay here. 

So we now have our community models and can leverage MICOM fully by simulating community growth - let's discuss what we want to look at. 

### Comparative Metabolomics

Now that our models are ready to go, we can think about some of the insights we might gain from these samples. First and foremost, we can investigate the metabolomic response of the gut microbiome in a set of individuals from underrepresented groups in the microbiome - in these different groups, do the microbes fall into different niche spaces?

Additionally, we can use MICOM to take a mechanistic look at how the microbiome of these individuals changes based on __dietary input__. That is, if we switch the diet of individuals from indigenous populations from their own traditional diet to an "average Western Diet", how does the metabolomic profile of the microbiome shift?

All that and more, coming up. Stay tuned!




First, we can download a sample of the "average western diet". 

In [ ]:
!wget -O western_diet.qza https://zenodo.org/record/3755182/files/western_diet_gut.qza?download=1

This is again a QIIME 2 artifact, which we can load into MICOM. Let's peek at what this diet looks like: 

In [ ]:
from micom.qiime_formats import load_qiime_medium

western_medium = load_qiime_medium("western_diet.qza")
western_medium.flux = western_medium.flux

Many dietary components get absorbed in the small intestine. This medium was 
created by taking dietary components and depleting all nutrients absorbed in the small intestine by a factor of 10 (indicated by the dilution column). 


Next, we'll download the traditional diet for each of our communities that we constructed by hand. Since this is a .csv file, we can read it using pandas. We'll use both these media in the coming steps. 

In [ ]:
traditional_medium = pd.read_csv('data/per_sample_media.csv')
traditional_medium['flux'] = traditional_medium['flux']/10

You'll notice that this dataframe is much longer - that's because this medium is "per-sample", meaning each of the nine models has it's own medium represented in the file, dependent on the traditional food of the community they live in. 

### Growing the models
Great, now we have our media & our models, it's time to get growing. We'll grow our nine samples each twice, once on the "average western diet" and once on the traditional diet we constructed. This will take some time, so we'll use this opportunity to discuss more in depth what these processes do, and what to look for in the results. First, let's run the `grow()` command using the medium representing the average western diet. This will take the models we've built,and find an optimal solution to the fluxes based upon the medium that's been applied. 

In [ ]:
from micom.workflows import grow
import pickle

western_growth = grow(manifest, "models", western_medium, tradeoff=0.5, threads=2)

# We'll save the results to a file
pickle.dump(western_growth, open("western_growth.pickle", "wb"))

If that takes too long or was aborted, we can read it in from the treasure chest.

In [ ]:
# Will only run if the previous step failed

import pickle

try:
  western_growth
except NameError:
  western_growth = pickle.load(open("treasure_chest/western_growth.pickle", "rb"))

Now we'll do the same, but with the constructed medium representing the diet of these indigenous communities. 

In [ ]:
from micom.workflows import grow
import pickle

traditional_growth = grow(manifest, "models", traditional_medium, tradeoff=0.5, threads=2)

# We'll save the results to a file
pickle.dump(traditional_growth, open("treasure_chest/traditional_growth.pickle", "wb"))

Again, if that takes too long or was aborted, we can read it in from the treasure chest.

In [ ]:
# Will only run if the previous step failed

import pickle

try:
  traditional_growth
except NameError:
  traditional_growth = pickle.load(open("treasure_chest/traditional_growth.pickle", "rb"))

What kind of results did we get? Well, `grow` returns a tuple of 3 data sets:

1. The predicted growth rate for all taxa in all samples
2. The import and export fluxes for each taxon and the external environment
3. Annotations for the fluxes mapping to other databases

The growth rates are pretty straightforward.

In [ ]:
western_growth.growth_rates

We can compare these growth rates to those of the second growth simulation

In [ ]:
traditional_growth.growth_rates

### ↔️ Exchange Fluxes 

More interesting are the exchange fluxes. These reactions represent the import and export of metabolites from the system Let's look at those now

In [ ]:
western_growth.exchanges

And for the other group:

In [ ]:
traditional_growth.exchanges

So we see how much of each metabolite is either consumed or produced by each taxon in each sample. `tolerance` denotes the accuracy of the solver and tells you the smallest absolute flux that is likely different form zero (i.e. substantial flux). *All of the fluxes are normalized to 1g dry weight of bacteria*. So, you can directly compare fluxes between taxa, even if they are present at very different abundances. 

If you're curious what the abbreviation for each of these metabolites represents, that can be found in the annotations dataframe. For instance, let's find out what `"but[e]"` represents. 

In [ ]:
anns = western_growth.annotations
anns[anns.metabolite == "but[e]"]

Butyrate! Interesting, that's an important metabolite! All of these annotations and more information at are also available at https://vmh.life maintained by Dr. Ines Thiele's lab. 


# 📊 Visualizations

Let's put our these data output to use by visualizing the results. Because of the rich output of these models, it can be overwhelming to represent it all, but don't worry! There are tools in place for this already. 

We will use the standard visualizations included in MICOM. These tools take in the growth results we obtained before and create visualizations in standalone HTML files that bundle the plots and raw data and can be viewed directly in your browser.

First, let's look at the growth rate of each taxon. 

In [ ]:
from micom.viz import *

viz_western = plot_growth(western_growth, filename="western_growthrates.html")
viz_traditional = plot_growth(traditional_growth, filename="traditional_growthrates.html")

Normally, we could call `viz.view()` afterwards and it would open it in our web browser. However, this will not work in Colab. However, the plot function create the file `growth_rates_[DATE].html` in your `materials` folder. To open it simply download that file and view it in your browser. We can see that there are many things going on, but it's not super clear. Let's continue.

## Growth niches

One thing we can look at is whether individual taxa inhabit different growth niches between dietary contexts. Here we can use the `plot_exchanges_per_taxon` function to see how exchanges differ between taxa, within the groups. 

In [ ]:
plot_exchanges_per_taxon(western_growth, perplexity=4, direction="import", filename="western_niche.html")
plot_exchanges_per_taxon(traditional_growth, perplexity=4, direction="import", filename="traditional_niche.html")


This function projects the full set of import or export fluxes onto a two dimensional plane, and arranges taxa so that more similar flux patterns lie close together. Thus, taxa closer to one another compete for a more similar set of resources (and/or produce a more similar set of metabolites). The center of the plot signifies a more competitive nutrient space, whereas clusters on the outskirts denote more isolated niches.

You can tune [TSNE parameters](https://distill.pub/2016/misread-tsne/), such as perplexity, to get a more meaningful grouping. We will lower the perplexity here since we don't have enough data.


## Comparative Metabolomics

Now let's compare the metabolomic exports between the two dietary contexts. We're interested to see how the metabolomic profile of the microbiome changes when the diet changes, as changes in diet can lead to changes in health state.  To look into this deeper, we'll concatenate the exchange results into one dataframe, transform the data and then plot the metabolite exports on a heatmap. 

First let's annotate each dataframe with it's medium

In [ ]:
western_growth.exchanges['diet'] = 'western'
traditional_growth.exchanges['diet'] = 'traditional'

Now we'll concatenate the dataframes. MICOM fluxes are returned as normalized values, so we'll have to multiply the exchange flux by the corresponding taxonomic abundance. We'll also transform the data using a centered log ratio, to compare all the fluxes against each other. 

In [ ]:
import pandas as pd
import numpy as np

exports = pd.concat([western_growth.exchanges, traditional_growth.exchanges])
exports = exports[exports.direction == "export"].groupby(["diet","sample_id", "metabolite", "reaction"]).apply(
    lambda df: sum(df.flux * df.abundance)).reset_index().rename(columns = {0:'flux'}) #multiply flux by abundance
exports = pd.pivot_table(exports, index = ['diet','sample_id'], columns = 'metabolite', values = 'flux')
exports = exports.T.fillna(0.0)
exports = exports.apply(
    lambda xs: np.log(xs + 0.001) - np.log(xs.mean() + 0.001),
    axis=1)
exports = exports.reindex(exports.sum(axis = 1).sort_values().index)

We can use seaborn to plot our heatmap

In [ ]:
import seaborn as sns 
import numpy as np

sns.set(rc = {'figure.figsize':(15,30)})
sns.clustermap(exports.head(50), cmap = 'viridis')

# 🏫 Exercises

Up to now, we have mostly used MICOM's "high-level" API, which is designed for working with several samples in parallel. However, MICOM also allows you to work with single models. We will choose a single sample now for further analysis.

First, let's recall what samples we had. 

In [ ]:
manifest

## Diet-Specific Interactions
We can use the `load_pickle` funciton in MICOM to load one of the models we're interested in, and look at differences in growth between the two diets we have available. 

In [ ]:
from micom import load_pickle
from micom.qiime_formats import load_qiime_medium

traditional_medium = pd.read_csv("data/per_sample_media.csv")
traditional_medium = traditional_medium[traditional_medium['sample_id'].str.contains('chepang1')]
traditional_medium.index = traditional_medium.reaction

com = load_pickle("models/chepang1.pickle")
com.medium = traditional_medium.flux
sol = com.cooperative_tradeoff(fraction=0.5, fluxes=True, pfba=True)
sol

Go ahead and try growing this model using the other available medium (average western diet)

In [ ]:
# Your code here

In [ ]:
# Your code here

# 🔵 Addendum


## Choosing a tradeoff value

Even if you don't have growth rates available you can still use your data to choose a decent tradeoff value. This can be done by choosing the largest tradeoff value that still allows growth for the majority of the taxa that you observed in the sample (if they are present at an appreciable abundance, they should be able to grow). This can be done with the `tradeoff` workflow in MICOM that will run cooperative tradeoff with varying tradeoff values, which can be visualized with the `plot_tradeoff` function.

In [ ]:
from micom.workflows import tradeoff
import micom

tradeoff_results = tradeoff(manifest, "models", medium, threads=2)
tradeoff_results.to_csv("tradeoff.csv", index=False)

plot_tradeoff(tradeoff_results, tolerance=1e-4)

After opeing `tradeoff_[DATE].html` you will see that, for our example here, all tradeoff values work great. This is because we modeled very few taxa, which keeps the compettion down. If you would allow for fewer abundant taxa in the models, this would change drastically. For instance, here is an example from a colorectal cancer data set:

[![tradeoff example](https://micom-dev.github.io/micom/_images/tradeoff.png)](https://micom-dev.github.io/micom/_static/tradeoff.html)

You can see how not using the cooperative tradeoff would give you nonsense results where only 10% of all observed taxa grew. A tradeoff value of 0.6-0.8 would probably be a good choice for this particular data set.